# Refine labels for NK cells

In this notebook, we load all CD56dim and CD56bright NK cells across our subjects to refine final L3 labels. We'll combine, recluster, and assign final labels by taking the most frequent AIFI_L3 label in each cluster, and back-propogate those L3 labels to L2 and L1 based on our cell type hierarcy.

We'll also generate metadata, UMAP coordinates, and marker gene summaries for review of our final labels, then store all of the outputs in HISE for later use.

Because there are many NK cells, we'll divide them into smaller subsets based on cohort, sex, and visit grouping (defined below). Unlike some other cell types, we'll combine CMV-positive and CMV-negative samples to enhance our ability to detect CMV-responsive populations. We'll then review each subset and assemble all labeled data in later notebooks.

## Load packages

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
from datetime import date
import hisepy
import os
import pandas as pd
import scanpy as sc
import scanpy.external as sce
import tarfile

In [2]:
out_dir = 'output'
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

In [3]:
class_name = 'nk_cell'

## Helper functions

### HISE data
These functions make it easy to utilize files from the HISE cache

In [4]:
def cache_uuid_path(uuid):
    cache_path = '/home/jupyter/cache/{u}'.format(u = uuid)
    if not os.path.isdir(cache_path):
        hise_res = hisepy.reader.cache_files([uuid])
    filename = os.listdir(cache_path)[0]
    cache_file = '{p}/{f}'.format(p = cache_path, f = filename)
    return cache_file

In [5]:
def read_csv_uuid(uuid):
    cache_file = cache_uuid_path(uuid)
    res = pd.read_csv(cache_file)
    return res

In [6]:
def read_adata_uuid(uuid):
    cache_file = cache_uuid_path(uuid)
    res = sc.read_h5ad(cache_file)
    return res

In [7]:
def read_obs_uuid(uuid):
    cache_file = cache_uuid_path(uuid)
    res = sc.read_h5ad(cache_file, backed = 'r')
    obs = res.obs.copy()
    return obs

### Label refinement

This function uses the most frequent label within each cluster to refine label assignments.

In [8]:
def single_value(series):
    res = []
    for value in series:
        if isinstance(value, list):
            res.append(value[0])
        else:
            res.append(value)
    return res

In [9]:
def assign_most_frequent(adata, clusters, labels, keep_original = False, original_prefix = 'predicted_'):
    obs = adata.obs
    
    most_frequent_labels = (
        adata.obs
        .groupby(clusters)[labels]
        .agg(pd.Series.mode)
        .to_frame()
        .reset_index()
    )
    most_frequent_labels[labels] = single_value(most_frequent_labels[labels])
    
    if keep_original:
        obs = obs.rename({labels: original_prefix + labels}, axis = 1)
    else:
        obs = obs.drop(labels, axis = 1)
    
    obs = obs.merge(most_frequent_labels, on = clusters, how = 'left')
    
    adata.obs = obs
    
    return adata

This function back-propagates hierarchical labeling from AIFI_L3 back to AIFI_L2 and AIFI_L1 to ensure our labels agree across levels of our cell type hierarchy.

In [10]:
def propagate_hierarchy(
    adata,
    hierarchy_df,
    from_level = 'AIFI_L3',
    to_levels = ['AIFI_L2', 'AIFI_L1'],
    keep_original = True,
    original_prefix = 'predicted_'
):
    obs = adata.obs
    
    for to_level in to_levels:
        prop_df = hierarchy_df[[from_level, to_level]]
        prop_df = prop_df.drop_duplicates()
        
        if keep_original:
            obs = obs.rename({to_level: original_prefix + to_level}, axis = 1)
        else:
            obs = obs.drop(to_level, axis = 1)

        obs[from_level] = obs[from_level].astype(str)
        obs = obs.merge(prop_df, on = from_level, how = 'left')
        obs[from_level] = obs[from_level].astype('category')
        obs[to_level] = obs[to_level].astype('category')
    
    adata.obs = obs

    return adata

### Review outputs

These functions are used to assemble marker gene expression tables for review

In [11]:
def marker_frac_df(adata, markers, clusters = 'louvain_2'):
    gene_cl_frac = sc.pl.dotplot(
        adata, 
        groupby = clusters,
        var_names = markers,
        return_fig = True
    ).dot_size_df
    return gene_cl_frac

def marker_mean_df(adata, markers, log = False, clusters = 'louvain_2'):
    gene_cl_mean = sc.pl.dotplot(
        adata, 
        groupby = clusters,
        var_names = markers,
        return_fig = True,
        log = log
    ).dot_color_df
    
    return gene_cl_mean

def tidy_marker_df(adata, markers, clusters = 'louvain_2'):
    gene_cl_frac = marker_frac_df(adata, markers, clusters)
    gene_cl_frac = gene_cl_frac.reset_index(drop = False)
    gene_cl_frac = pd.melt(gene_cl_frac, id_vars = clusters, var_name = 'gene', value_name = 'gene_frac')
    
    gene_cl_mean = marker_mean_df(adata, markers, log = False, clusters = clusters)
    gene_cl_mean = gene_cl_mean.reset_index(drop = False)
    gene_cl_mean = pd.melt(gene_cl_mean, id_vars = clusters, var_name = 'gene', value_name = 'gene_mean')

    marker_df = gene_cl_frac.merge(gene_cl_mean, on = [clusters, 'gene'], how = 'left')
    return marker_df

This function retrieves both observations and UMAP coordinates in a single table for review.

In [12]:
def obs_with_umap(adata):
    obs = adata.obs
    
    umap_mat = adata.obsm['X_umap']
    umap_df = pd.DataFrame(umap_mat, columns = ['umap_1', 'umap_2'])
    obs['umap_1'] = umap_df['umap_1']
    obs['umap_2'] = umap_df['umap_2']

    return obs

This function applies data analysis methods to our scRNA-seq data, including normalization, HVG selection, PCA, nearest neighbors, UMAP, and Leiden clustering.

In [13]:
def process_adata(adata):
    
    # Keep a copy of the raw data
    adata = adata.raw.to_adata()
    adata.raw = adata

    print('Normalizing', end = "; ")
    # Normalize and log transform
    sc.pp.normalize_total(adata)
    sc.pp.log1p(adata)

    print('Finding HVGs', end = "; ")
    # Restrict downstream steps to variable genes
    sc.pp.highly_variable_genes(adata)
    adata = adata[:, adata.var_names[adata.var['highly_variable']]].copy()

    print('Scaling', end = "; ")
    # Scale variable genes
    sc.pp.scale(adata)

    print('PCA', end = "; ")
    # Run PCA
    sc.tl.pca(adata, svd_solver = 'arpack')

    print('Harmony', end = "; ")
    # Integrate subjects
    sce.pp.harmony_integrate(
        adata, 
        'subject.subjectGuid',
        max_iter_harmony = 30,
        verbose = False
    )
    
    print('Neighbors', end = "; ")
    # Find nearest neighbors
    sc.pp.neighbors(
        adata, 
        n_neighbors = 50,
        n_pcs = 30,
        use_rep = 'X_pca_harmony'
    )

    print('Leiden', end = "; ")
    # Find clusters
    sc.tl.leiden(
        adata, 
        resolution = 2, 
        key_added = 'leiden_2',
        n_iterations = 2
    )

    print('UMAP', end = "; ")
    # Run UMAP
    sc.tl.umap(adata, min_dist = 0.05)
    
    print('Renormalizing')
    adata = adata.raw.to_adata()
    adata.raw = adata

    # Normalize and log transform
    sc.pp.normalize_total(adata)
    sc.pp.log1p(adata)
    
    return adata

## Cell Type Markers

These are the set of marker genes that we'll use to review our cell type labels.

In [14]:
l2_markers = [
    'FCGR3A', 'MKI67', 'NCAM1'
]

l3_markers = [
    'FCER1G', 'FCGR3A', 'GZMK', 'KLRC2',
    'MKI67', 'MX1', 'NCAM1', 'ZBTB16'
]

## Read cell type hierarchy from HISE

As part of label refinement, we'll back-propagate our cell type labels from refined AIFI_L3 labels to their parent cell classes at AIFI_L2 and AIFI_L1. To do this, we need the hierarchical relationships between these levels, which have been generated for our cell type reference dataset.

In [15]:
hierarchy_uuid = '1a44252c-8cab-4c8f-92c9-d8f3af633790'
hierarchy_df = read_csv_uuid(hierarchy_uuid)

## Read sample metadata from HISE

We previously assembled sample metadata and CMV status for each subject. We'll retrieve and combine these to utilize for selecting subsets of samples.

In [16]:
sample_meta_uuid = 'd82c5c42-ae5f-4e67-956e-cd3b7bf88105'
sample_meta = read_csv_uuid(sample_meta_uuid)

In [17]:
sample_meta.shape

(868, 32)

In [18]:
cmv_meta_uuid = '9469f67c-b09a-454d-9fb9-f50ff3494d69'
cmv_path = cache_uuid_path(cmv_meta_uuid)
cmv_meta = pd.read_csv(cmv_path, index_col = 0)
cmv_meta = cmv_meta.drop_duplicates()

In [19]:
cmv_meta.shape

(96, 4)

In [20]:
sample_meta = sample_meta.merge(cmv_meta, on = 'subject.subjectGuid', how = 'left')

In [21]:
sample_meta.shape

(868, 35)

## Assign sample groups

To subdivide the full set of cells, we'll use groups that include cohort, sex, and a subset of visits. To group our visit data, we'll define 3 visit groups, and use those together with the other metadata to group samples.

In [22]:
visit_group_dict = {
    'Flu Year 1 Day 0': 'Year 1',
    'Flu Year 1 Day 7': 'Year 1',
    'Flu Year 1 Day 90': 'Year 1',
    'Flu Year 1 Stand-Alone': 'Year 1',
    'Flu Year 2 Day 0': 'Year 2',
    'Flu Year 2 Day 7': 'Year 2',
    'Flu Year 2 Day 90': 'Year 2',
    'Flu Year 2 Stand-Alone': 'Year 2',
    'Immune Variation Day 0': 'Immune Variation',
    'Immune Variation Day 7': 'Immune Variation',
    'Immune Variation Day 90': 'Immune Variation',
    'Flu Year 3 Stand-Alone': 'Immune Variation',
}

In [23]:
visit_groups = list(set(visit_group_dict.values()))

In [24]:
sample_meta['sample.visitGroup'] = [visit_group_dict[v] for v in sample_meta['sample.visitName']]

In [25]:
group_samples_by = ['cohort.cohortGuid', 'subject.biologicalSex']

In [26]:
grouped_meta = sample_meta.groupby(group_samples_by)

In [27]:
split_meta = {}
for group_tuple, meta in grouped_meta:
    split_name = '_'.join(group_tuple)
    split_meta[split_name] = meta

## Identify files in HISE

For this analysis, we'll read in these files from HISE storage from previous steps. We'll group these into "large" files, which are from the set of 5 very large cell type assignments, and "small" files, which are from other cell types.

In [28]:
large_uuids = {
    'BR1_Female': {
        'BR1_Female_Negative_CD56dim-NK-cell': 'd35f90dd-1ce4-4743-b07f-d13caa6b271e',
        'BR1_Female_Positive_CD56dim-NK-cell': '994c0e01-c7a5-4533-bc55-dab04f880ae7'
    },
    'BR1_Male': {
        'BR1_Male_Negative_CD56dim-NK-cell': '40516b72-5052-49b1-97df-8821d52d7893',
        'BR1_Male_Positive_CD56dim-NK-cell': '08411747-d5a1-4d81-8b82-6e64f18f497b'
    },
    'BR2_Female': {
        'BR2_Female_Negative_CD56dim-NK-cell': 'cbd07e76-7365-402f-bf5a-dae364bf6084',
        'BR2_Female_Positive_CD56dim-NK-cell': 'd0ffe8cf-cdc4-4890-a9d1-665437a9bc25'
    },
    'BR2_Male': {
        'BR2_Male_Negative_CD56dim-NK-cell': '1b839052-9906-41f6-b50a-41cbac1b2948',
        'BR2_Male_Positive_CD56dim-NK-cell': 'aa29462f-f39b-43b7-93fc-df475ccf94d6'
    }
}
small_uuids = {
    'CD56bright-NK-cell': '313e5417-1996-4592-86e7-9d3f5604fdc7'
}

## Read data from small sets

Now, we can read in the data from small sets to memory so that splitting them up for analysis is more efficient.

For large sets, we'll read files in selectively based on their metadata group to limit the amount of memory we utilize.

In [29]:
small_adata_dict = {}
for group_name, uuid in small_uuids.items():
    small_adata_dict[group_name] = read_adata_uuid(uuid)

In [30]:
small_adata = sc.concat(small_adata_dict)

In [31]:
small_group_adata = {}
for grouping, meta in split_meta.items():
    keep_cells = small_adata.obs['sample.sampleKitGuid'].isin(meta['sample.sampleKitGuid'])
    group_adata = small_adata[keep_cells]
    small_group_adata[grouping] = group_adata

In [32]:
del small_adata

## Read and process data per group

Here, we'll step through each group based on cohort, sex, and CMV, assemble all related data across our selected files for this cell type, then split the results back out to multiple files based on the Visit Groups defined above.

This way, we can combine across L2 cell classes without generating enormous datasets (almost 6M cells for CD4 T cells alone, for example).

Once split up, we'll reprocess each subset of data to generate nice clusters and UMAP projections, then refine L3 cell type labels by taking the most frequent label within each cluster. After refining labels, we'll propagate labels back to L2 and L1 so they're consistent.

Finally, we'll output these refined results per group, as well as a .csv file with updated labels to enable review of our analysis.

In [33]:
for grouping, meta in split_meta.items():
    print(grouping)

    # Check if output files have been generated previously
    out_files = {}
    out_exists = []
    for visit_group in visit_groups:
        out_file = 'output/diha_{c}_{g}_{v}_AIFI_L3_review_{d}.h5ad'.format(
            c = class_name,
            g = grouping,
            v = visit_group,
            d = date.today()
        )
        out_files[visit_group] = [out_file]
        out_exists.append(os.path.isfile(out_file))
    
    if sum(out_exists) == len(visit_groups):
        print('{g} Previously processed. Skipping.'.format(g = grouping))
    else:
    
        all_adata = small_group_adata[grouping]
        print('Small types: {n} cells'.format(n = all_adata.shape[0]))
        
        # Read Large Files
        large_group_uuids = large_uuids[grouping]
        for group_name, uuid in large_group_uuids.items():
            group_adata = read_adata_uuid(uuid)
            print('{g}: {n} cells'.format(g = group_name, n = group_adata.shape[0]))
            all_adata = sc.concat([all_adata, group_adata])
            del group_adata
        print('Total: {n} cells'.format(n = all_adata.shape[0]))
    
        visit_group_meta = meta.groupby('sample.visitGroup')

        for (visit_group, visit_meta) in visit_group_meta:
            out_file = out_files[visit_group][0]
            keep_cells = all_adata.obs['sample.sampleKitGuid'].isin(visit_meta['sample.sampleKitGuid'])
            visit_group_adata = all_adata[keep_cells]
            print('>>> Processing {v}: {n} cells'.format(v = visit_group, n = visit_group_adata.shape[0]))
            
            visit_group_adata = process_adata(visit_group_adata)
            
            # Refine labels based on in-group clustering
            visit_group_adata = assign_most_frequent(
                visit_group_adata, 
                'leiden_2', 'AIFI_L3', 
                keep_original = True, 
                original_prefix = 'predicted_'
            )
            
            visit_group_adata = propagate_hierarchy(
                visit_group_adata,
                hierarchy_df,
                from_level = 'AIFI_L3',
                to_levels = ['AIFI_L2', 'AIFI_L1'],
                keep_original = True,
                original_prefix = 'predicted_'
            )
            
            # Save results for this visit group
            visit_group_adata.write_h5ad(out_file)

            # Save metadata and UMAP
            meta_csv = 'output/review/diha_{c}_{g}_{v}_AIFI_L3_review_meta_{d}.csv'.format(
                c = class_name,
                g = grouping,
                v = visit_group,
                d = date.today()
            )
            visit_group_obs = obs_with_umap(visit_group_adata)
            visit_group_obs.to_csv(meta_csv)
            out_files[visit_group].append(meta_csv)

            # Save marker expression summaries
            out_l2_markers = 'output/review/diha_{c}_{g}_{v}_AIFI_L2_review_markers_{d}.csv'.format(
                c = class_name,
                g = grouping,
                v = visit_group,
                d = date.today()
            )
            l2_marker_df = tidy_marker_df(
                visit_group_adata,
                l2_markers,
                'AIFI_L2'
            )
            l2_marker_df.to_csv(out_l2_markers)
            out_files[visit_group].append(out_l2_markers)

            out_l3_markers = 'output/review/diha_{c}_{g}_{v}_AIFI_L3_review_markers_{d}.csv'.format(
                c = class_name,
                g = grouping,
                v = visit_group,
                d = date.today()
            )
            l3_marker_df = tidy_marker_df(
                visit_group_adata,
                l3_markers,
                'AIFI_L3'
            )
            l3_marker_df.to_csv(out_l3_markers)
            out_files[visit_group].append(out_l3_markers)

BR1_Female
BR1_Female Previously processed. Skipping.
BR1_Male
BR1_Male Previously processed. Skipping.
BR2_Female
BR2_Female Previously processed. Skipping.
BR2_Male
BR2_Male Previously processed. Skipping.


## Assemble output files for upload

### h5ad files

In [44]:
output_files = os.listdir('output')
h5ad_files = []
for output_file in output_files:
    if class_name in output_file:
        h5ad_files.append('{d}/{f}'.format(d = 'output', f = output_file))

### Review files

In [45]:
rev_files = os.listdir('output/review')
review_files = ['output/review/{f}'.format(f = f) for f in rev_files]

### Combine metadata files to assemble a full set

In [46]:
meta_files = []
for review_file in review_files:
    if 'meta' in review_file:
        meta_files.append(review_file)

meta_list = []
for meta_file in meta_files:
    meta_list.append(pd.read_csv(meta_file, index_col = 0))
all_meta = pd.concat(meta_list)

In [47]:
meta_csv = 'output/diha_{c}_AIFI_L3_refinement_meta_{d}.csv'.format(c = class_name, d = date.today())
all_meta.to_csv(meta_csv)
meta_parquet = 'output/diha_{c}_AIFI_L3_refinement_meta_{d}.parquet'.format(c = class_name, d = date.today())
all_meta.to_parquet(meta_parquet)

### Bundle review files into a .tar for later use

In [48]:
review_tar = 'output/diha_{c}_AIFI_L3_refinement_review_{d}.tar.gz'.format(c = class_name, d = date.today())
tar = tarfile.open(review_tar, 'w:gz')
for review_file in review_files:
    tar.add(review_file)
tar.close()

## Upload assembled results to HISE

In [49]:
study_space_uuid = 'de025812-5e73-4b3c-9c3b-6d0eac412f2a'
title = 'DIHA NK cell AIFI_L3 Refinement {d}'.format(d = date.today())

In [50]:
in_files = []
for group_name, file_dict in large_uuids.items():
    for type_group, uuid in file_dict.items():
        in_files.append(uuid)
in_files

['d35f90dd-1ce4-4743-b07f-d13caa6b271e',
 '994c0e01-c7a5-4533-bc55-dab04f880ae7',
 '40516b72-5052-49b1-97df-8821d52d7893',
 '08411747-d5a1-4d81-8b82-6e64f18f497b',
 'cbd07e76-7365-402f-bf5a-dae364bf6084',
 'd0ffe8cf-cdc4-4890-a9d1-665437a9bc25',
 '1b839052-9906-41f6-b50a-41cbac1b2948',
 'aa29462f-f39b-43b7-93fc-df475ccf94d6']

In [51]:
out_files = h5ad_files + [meta_csv, meta_parquet, review_tar]

In [52]:
out_files

['output/diha_nk_cell_BR2_Female_Year 2_AIFI_L3_review_2024-03-22.h5ad',
 'output/diha_nk_cell_BR2_Female_Year 1_AIFI_L3_review_2024-03-22.h5ad',
 'output/diha_nk_cell_BR1_Female_Year 1_AIFI_L3_review_2024-03-22.h5ad',
 'output/diha_nk_cell_BR1_Male_Year 2_AIFI_L3_review_2024-03-22.h5ad',
 'output/diha_nk_cell_BR1_Male_Immune Variation_AIFI_L3_review_2024-03-22.h5ad',
 'output/diha_nk_cell_BR2_Male_Year 1_AIFI_L3_review_2024-03-22.h5ad',
 'output/diha_nk_cell_BR2_Male_Immune Variation_AIFI_L3_review_2024-03-22.h5ad',
 'output/diha_nk_cell_BR1_Male_Year 1_AIFI_L3_review_2024-03-22.h5ad',
 'output/diha_nk_cell_BR2_Female_Immune Variation_AIFI_L3_review_2024-03-22.h5ad',
 'output/diha_nk_cell_BR1_Female_Immune Variation_AIFI_L3_review_2024-03-22.h5ad',
 'output/diha_nk_cell_BR1_Female_Year 2_AIFI_L3_review_2024-03-22.h5ad',
 'output/diha_nk_cell_BR2_Male_Year 2_AIFI_L3_review_2024-03-22.h5ad',
 'output/diha_nk_cell_AIFI_L3_refinement_meta_2024-03-22.csv',
 'output/diha_nk_cell_AIFI_L3_ref

In [53]:
hisepy.upload.upload_files(
    files = out_files,
    study_space_id = study_space_uuid,
    title = title,
    input_file_ids = in_files
)

output/diha_nk_cell_BR2_Female_Year 2_AIFI_L3_review_2024-03-22.h5ad
output/diha_nk_cell_BR2_Female_Year 1_AIFI_L3_review_2024-03-22.h5ad
output/diha_nk_cell_BR1_Female_Year 1_AIFI_L3_review_2024-03-22.h5ad
output/diha_nk_cell_BR1_Male_Year 2_AIFI_L3_review_2024-03-22.h5ad
output/diha_nk_cell_BR1_Male_Immune Variation_AIFI_L3_review_2024-03-22.h5ad
output/diha_nk_cell_BR2_Male_Year 1_AIFI_L3_review_2024-03-22.h5ad
output/diha_nk_cell_BR2_Male_Immune Variation_AIFI_L3_review_2024-03-22.h5ad
output/diha_nk_cell_BR1_Male_Year 1_AIFI_L3_review_2024-03-22.h5ad
output/diha_nk_cell_BR2_Female_Immune Variation_AIFI_L3_review_2024-03-22.h5ad
output/diha_nk_cell_BR1_Female_Immune Variation_AIFI_L3_review_2024-03-22.h5ad
output/diha_nk_cell_BR1_Female_Year 2_AIFI_L3_review_2024-03-22.h5ad
output/diha_nk_cell_BR2_Male_Year 2_AIFI_L3_review_2024-03-22.h5ad
output/diha_nk_cell_AIFI_L3_refinement_meta_2024-03-22.csv
output/diha_nk_cell_AIFI_L3_refinement_meta_2024-03-22.parquet
output/diha_nk_cell_AI

 1


you are trying to upload file_ids... ['output/diha_nk_cell_BR2_Female_Year 2_AIFI_L3_review_2024-03-22.h5ad', 'output/diha_nk_cell_BR2_Female_Year 1_AIFI_L3_review_2024-03-22.h5ad', 'output/diha_nk_cell_BR1_Female_Year 1_AIFI_L3_review_2024-03-22.h5ad', 'output/diha_nk_cell_BR1_Male_Year 2_AIFI_L3_review_2024-03-22.h5ad', 'output/diha_nk_cell_BR1_Male_Immune Variation_AIFI_L3_review_2024-03-22.h5ad', 'output/diha_nk_cell_BR2_Male_Year 1_AIFI_L3_review_2024-03-22.h5ad', 'output/diha_nk_cell_BR2_Male_Immune Variation_AIFI_L3_review_2024-03-22.h5ad', 'output/diha_nk_cell_BR1_Male_Year 1_AIFI_L3_review_2024-03-22.h5ad', 'output/diha_nk_cell_BR2_Female_Immune Variation_AIFI_L3_review_2024-03-22.h5ad', 'output/diha_nk_cell_BR1_Female_Immune Variation_AIFI_L3_review_2024-03-22.h5ad', 'output/diha_nk_cell_BR1_Female_Year 2_AIFI_L3_review_2024-03-22.h5ad', 'output/diha_nk_cell_BR2_Male_Year 2_AIFI_L3_review_2024-03-22.h5ad', 'output/diha_nk_cell_AIFI_L3_refinement_meta_2024-03-22.csv', 'output/

(y/n) y


{'trace_id': '690e51d2-020e-4387-a4e1-6e5b140aedf2',
 'files': ['output/diha_nk_cell_BR2_Female_Year 2_AIFI_L3_review_2024-03-22.h5ad',
  'output/diha_nk_cell_BR2_Female_Year 1_AIFI_L3_review_2024-03-22.h5ad',
  'output/diha_nk_cell_BR1_Female_Year 1_AIFI_L3_review_2024-03-22.h5ad',
  'output/diha_nk_cell_BR1_Male_Year 2_AIFI_L3_review_2024-03-22.h5ad',
  'output/diha_nk_cell_BR1_Male_Immune Variation_AIFI_L3_review_2024-03-22.h5ad',
  'output/diha_nk_cell_BR2_Male_Year 1_AIFI_L3_review_2024-03-22.h5ad',
  'output/diha_nk_cell_BR2_Male_Immune Variation_AIFI_L3_review_2024-03-22.h5ad',
  'output/diha_nk_cell_BR1_Male_Year 1_AIFI_L3_review_2024-03-22.h5ad',
  'output/diha_nk_cell_BR2_Female_Immune Variation_AIFI_L3_review_2024-03-22.h5ad',
  'output/diha_nk_cell_BR1_Female_Immune Variation_AIFI_L3_review_2024-03-22.h5ad',
  'output/diha_nk_cell_BR1_Female_Year 2_AIFI_L3_review_2024-03-22.h5ad',
  'output/diha_nk_cell_BR2_Male_Year 2_AIFI_L3_review_2024-03-22.h5ad',
  'output/diha_nk_cell

In [54]:
import session_info
session_info.show()